In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import time

from PIL import Image
from IPython.display import display

import tensorflow as tf
from tensorflow.keras import layers, models

from build_model_ed import build_model_1, build_model_2, build_model_3
from build_model_lstm import build_model_lstm_1, build_model_lstm_2, build_model_lstm_3
from get_frames import get_frames
from get_encoder_decoder import get_encoder_decoder
from get_errors import get_ssim, get_mse, get_errors
from assemble_image import assemble_image

In [ ]:
#gpu check
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
#load data
test_size = 0.1
stride_step = 50
window_size = 5
frames_to_predict=window_size
norm_factor=1
dtype = 'uint8'

#filename = f'save_data/e_d_train_all_str_{stride_step}_ts_{test_size}_{dtype}_norm_{norm_factor}.pkl'
filename = f'save_data/e_d_test_all_str_{stride_step}_ts_{test_size}_{dtype}_norm_{norm_factor}.pkl'
#load data
with open(filename, 'rb') as f:
    image_list,frames_data = pickle.load(f)
    
#get input shape for encoder-decoder model
input_shape_e_d = frames_data.shape[1:]
del frames_data 

print(f'Image files: {image_list}')
print(f'Encoder-decoder input shape: {input_shape_e_d}')

In [ ]:
unit_numb = 5
#lstm settings
cells_list = [10, 10]
many_to_many = True
ifSummary = True
ifEncoder=True
ifDecoder=True

ifDense = True
input_shape = (None, unit_numb)
cells_list_str = '_'.join(str(x) for x in cells_list)
model_ed_name = f'model_ed_2_ps_2_bn_True_du_{unit_numb}'
model_lstm_name = f'model_lstm_window_units_{unit_numb}_bs2_{input_shape[-1]}_{cells_list_str}_ifDense_{ifDense}'
#model_lstm_name = f'model_lstm_window_{window_size}_units_{unit_numb}_bs2_{input_shape[-1]}_{cells_list_str}_ifDense_{ifDense}'


print(f'\nModel encoder-decoder name: {model_ed_name}')
print(f'\nModel lstm name: {model_lstm_name}')

In [ ]:
#load models
print(f'\nModel name: {model_ed_name}\n\n')

loss = 'mean_squared_error'
optimizer = tf.keras.optimizers.legacy.Adadelta(learning_rate=0.1, name="Adadelta")

#input_shape = frames_data.shape[1:]
input_shape = input_shape_e_d

filename = f'saved_models/{model_ed_name}_settings.pkl'
#load nn settings
with open(filename, 'rb') as f:
    dense_units,ifBatchNorm,kernel_size,nn_blocks = pickle.load(f)
model_settings = (input_shape, dense_units,ifBatchNorm,kernel_size,nn_blocks)
model_weights_path = f'saved_models/stride_{stride_step}/{model_ed_name}/{model_ed_name}'

model_ed_list = get_encoder_decoder(
    model_ed_name, 
    model_weights_path, 
    model_settings, 
    loss, optimizer, 
    ifEncoder=ifEncoder, 
    ifDecoder=ifDecoder, 
    ifSummary=ifSummary
)

In [ ]:
model_lstm_name

In [ ]:
#load lstm model
lstm_folder = 'lstm_window_many_to_many'

print(f'\nModel name: {model_lstm_name}')

# extract network parameters from model name
pars = model_lstm_name.split('_')
if 1:
    input_shape = (None, int(pars[4]))
    ifDense = eval(pars[-1])
    cells_list = [int(x) for x in pars[7:-2]]
else:
    input_shape = (None, int(pars[5]))
    ifDense = eval(pars[-1])
    cells_list = [int(x) for x in pars[8:-2]]

loss = 'mean_squared_error'
optimizer = tf.keras.optimizers.legacy.Adadelta(learning_rate=0.1, name="Adadelta")
model_lstm = build_model_lstm_3(input_shape, cells_list, ifDense=ifDense, ifDropout=False, many_to_many=many_to_many)
model_lstm.compile(loss=loss, 
              optimizer=optimizer, )

if ifSummary:
    model_lstm.summary() 


model_lstm.load_weights(f'saved_models/{lstm_folder}/stride_{stride_step}/{model_lstm_name}/{model_lstm_name}').expect_partial()  

In [ ]:
image_list

In [ ]:
#choose image
test_case = image_list[5]
stride_step = 50


#get frames from the image
path_list = ['PuckerImages//RGB_cropped',
             'TwistImages//RGB',
             'FoldImages//RGB',]

print(f'Chosen image: {test_case}')
for path in path_list:
    if test_case[:4]==path[:4]:
        break
        
img = Image.open(f'{path}//{test_case}')
image_data_gt = np.array(img, dtype=dtype)
#crop the end
image_data_gt = image_data_gt[:,:-96,:]
print(f'Image shape: {image_data_gt.shape}')
#get frames
image_data_frames_gt = get_frames(image_data_gt, stride_step, ifPrint = False)
print(f'Framed image shape: {image_data_frames_gt.shape}')

In [ ]:
#unit 5
numb_cols = 3
numb_plots_per_frame = 3

number_of_frames = image_data_frames_gt.shape[0]
numb_rows = int(np.ceil(number_of_frames/numb_cols))

model_encoder = model_ed_list[0]
model_decoder = model_ed_list[1]


fig = plt.figure(figsize=(20, 15*numb_rows))
outer = gridspec.GridSpec(numb_rows, numb_cols, wspace=0.2, hspace=0.1)
linewidth = 3.0
fontsize_sec = 14
'''
the prediction is implemented through for loop as it is how the data is obtained in the real AFP process
input data is received in chuncks as tape moves
'''
frames_to_pred_total = number_of_frames-frames_to_predict
frames_to_pred_total = 70
for i in range(frames_to_pred_total):
    #reset lstm state to get independent prediction between timesteps (batches)
    model_lstm.reset_states()
    print(f'\n\n\nStep {i+1}')
    #get ground truth window
    image_data_frames_input = image_data_frames_gt[i:window_size+i]
    print(f'Input shape: {image_data_frames_input.shape}')
    
    print('\n##########Encoding started#############')
    image_data_frames_encoded = model_encoder.predict(image_data_frames_input)
    print('##########Encoding finished############')
    print(f'Encoded shape: {image_data_frames_encoded.shape}')
    
    print('\n##########LSTM prediction started#############')
    image_data_frames_lstm = np.copy(image_data_frames_encoded)
    lstm_input = image_data_frames_lstm
    #predict
    print(f'Input shape: {lstm_input.shape}')
    predicted_sequence = model_lstm.predict(lstm_input[np.newaxis])
    print(predicted_sequence.shape)
    image_data_frames_lstm = np.concatenate((image_data_frames_lstm, predicted_sequence[0]), axis=0)

    print(f'Output shape: {image_data_frames_lstm.shape}')
    print('##########LSTM prediction finished#############')

    print('\n##########Decoding started#############')
    image_data_frames_decoded = model_decoder.predict(image_data_frames_lstm)
    print('##########Decoding finished############')
    print(f'Decoded shape: {image_data_frames_decoded.shape}')

    #assemble the whole image from frames
    image_data_predicted = assemble_image(image_data_frames_decoded)
  
    #plot
    inner = gridspec.GridSpecFromSubplotSpec(
        numb_plots_per_frame, 1,
        subplot_spec=outer[i], wspace=0.1, hspace=0.1)

    #plot ground truth
    ax = plt.Subplot(fig, inner[0])
    x_lim = image_data_gt.shape[1]
    #plot only a portion of gt for comparison
    index_current = window_size*stride_step+stride_step*i+frames_to_predict*stride_step
    ax.imshow(image_data_gt[:,:index_current,:])
    ax.set_xlim(0, x_lim)
    ax.set_title(f'Step {i+1}, Ground truth')
    #plot separator input/prediction
    x_separator = window_size*stride_step+stride_step*i
    ax.axvline(x = x_separator, color = 'red')
    fig.add_subplot(ax)
    
    #plot prediction
    ax = plt.Subplot(fig, inner[1])
    #reassemble to include all previous gt
    image_data_predicted = np.concatenate(
        (
            image_data_gt[:,:x_separator,:],
            image_data_predicted[:,-frames_to_predict*stride_step:,:]
        ),
        axis=1
    )
    ax.imshow(image_data_predicted)
    ax.set_title(f'LSTM prediction')
    ax.set_xlim(0, x_lim)
    ax.axvline(x = x_separator, color = 'red')
    fig.add_subplot(ax)
    
    if 1:
        #get errors
        image_gt_1_frames = image_data_frames_gt[i+window_size:i+window_size+frames_to_predict]
        image_pred_2_frames = image_data_frames_decoded[window_size:]

        ssim_frames, mse_gray_frames, mse_rgb_frames = get_errors(
            image_gt_1_frames, 
            image_pred_2_frames
                )   
        #plot errors
        
        
        ax = plt.Subplot(fig, inner[2])
        ax.set_title(f'Errors')   
        ax.plot(mse_gray_frames, label='MSE grayscale')
        ax.plot(mse_rgb_frames, label='MSE RGB')
        color = 'tab:purple'
        ax_add = ax.twinx()
        ax_add.plot(ssim_frames, color=color, label='SSIM')
        ax_add.set_ylabel('SSIM', color=color)
        ax_add.tick_params(axis='y', labelcolor=color)
        lines, labels = ax.get_legend_handles_labels()
        lines2, labels2 = ax_add.get_legend_handles_labels()
        ax.legend(lines + lines2, labels + labels2)
        x_lim = number_of_frames
        ax.set_xlim(0, x_lim)
        fig.add_subplot(ax)
        fig.add_subplot(ax_add)


In [ ]:
#unit 5
numb_cols = 3
numb_plots_per_frame = 3

number_of_frames = image_data_frames_gt.shape[0]
numb_rows = int(np.ceil(number_of_frames/numb_cols))

model_encoder = model_ed_list[0]
model_decoder = model_ed_list[1]


fig = plt.figure(figsize=(20, 15*numb_rows))
outer = gridspec.GridSpec(numb_rows, numb_cols, wspace=0.2, hspace=0.1)
linewidth = 3.0
fontsize_sec = 14
'''
the prediction is implemented through for loop as it is how the data is obtained in the real AFP process
input data is received in chuncks as tape moves
'''
frames_to_pred_total = number_of_frames-frames_to_predict
frames_to_pred_total = 70
for i in range(frames_to_pred_total):
    #reset lstm state to get independent prediction between timesteps (batches)
    model_lstm.reset_states()
    print(f'\n\n\nStep {i+1}')
    #get ground truth window
    image_data_frames_input = image_data_frames_gt[i:window_size+i]
    print(f'Input shape: {image_data_frames_input.shape}')
    
    print('\n##########Encoding started#############')
    image_data_frames_encoded = model_encoder.predict(image_data_frames_input)
    print('##########Encoding finished############')
    print(f'Encoded shape: {image_data_frames_encoded.shape}')
    
    print('\n##########LSTM prediction started#############')
    image_data_frames_lstm = np.copy(image_data_frames_encoded)
    lstm_input = image_data_frames_lstm
    #predict
    print(f'Input shape: {lstm_input.shape}')
    predicted_sequence = model_lstm.predict(lstm_input[np.newaxis])
    print(predicted_sequence.shape)
    image_data_frames_lstm = np.concatenate((image_data_frames_lstm, predicted_sequence[0]), axis=0)

    print(f'Output shape: {image_data_frames_lstm.shape}')
    print('##########LSTM prediction finished#############')

    print('\n##########Decoding started#############')
    image_data_frames_decoded = model_decoder.predict(image_data_frames_lstm)
    print('##########Decoding finished############')
    print(f'Decoded shape: {image_data_frames_decoded.shape}')

    #assemble the whole image from frames
    image_data_predicted = assemble_image(image_data_frames_decoded)
  
    #plot
    inner = gridspec.GridSpecFromSubplotSpec(
        numb_plots_per_frame, 1,
        subplot_spec=outer[i], wspace=0.1, hspace=0.1)

    #plot ground truth
    ax = plt.Subplot(fig, inner[0])
    x_lim = image_data_gt.shape[1]
    #plot only a portion of gt for comparison
    index_current = window_size*stride_step+stride_step*i+frames_to_predict*stride_step
    ax.imshow(image_data_gt[:,:index_current,:])
    ax.set_xlim(0, x_lim)
    ax.set_title(f'Step {i+1}, Ground truth')
    #plot separator input/prediction
    x_separator = window_size*stride_step+stride_step*i
    ax.axvline(x = x_separator, color = 'red')
    fig.add_subplot(ax)
    
    #plot prediction
    ax = plt.Subplot(fig, inner[1])
    #reassemble to include all previous gt
    image_data_predicted = np.concatenate(
        (
            image_data_gt[:,:x_separator,:],
            image_data_predicted[:,-frames_to_predict*stride_step:,:]
        ),
        axis=1
    )
    ax.imshow(image_data_predicted)
    ax.set_title(f'LSTM prediction')
    ax.set_xlim(0, x_lim)
    ax.axvline(x = x_separator, color = 'red')
    fig.add_subplot(ax)
    
    if 1:
        #get errors
        image_gt_1_frames = image_data_frames_gt[i+window_size:i+window_size+frames_to_predict]
        image_pred_2_frames = image_data_frames_decoded[window_size:]

        ssim_frames, mse_gray_frames, mse_rgb_frames = get_errors(
            image_gt_1_frames, 
            image_pred_2_frames
                )   
        #plot errors
        
        
        ax = plt.Subplot(fig, inner[2])
        ax.set_title(f'Errors')   
        ax.plot(mse_gray_frames, label='MSE grayscale')
        ax.plot(mse_rgb_frames, label='MSE RGB')
        color = 'tab:purple'
        ax_add = ax.twinx()
        ax_add.plot(ssim_frames, color=color, label='SSIM')
        ax_add.set_ylabel('SSIM', color=color)
        ax_add.tick_params(axis='y', labelcolor=color)
        lines, labels = ax.get_legend_handles_labels()
        lines2, labels2 = ax_add.get_legend_handles_labels()
        ax.legend(lines + lines2, labels + labels2)
        x_lim = number_of_frames
        ax.set_xlim(0, x_lim)
        fig.add_subplot(ax)
        fig.add_subplot(ax_add)


In [ ]:
#unit 5
numb_cols = 3
numb_plots_per_frame = 3

number_of_frames = image_data_frames_gt.shape[0]
numb_rows = int(np.ceil(number_of_frames/numb_cols))

model_encoder = model_ed_list[0]
model_decoder = model_ed_list[1]


fig = plt.figure(figsize=(20, 15*numb_rows))
outer = gridspec.GridSpec(numb_rows, numb_cols, wspace=0.2, hspace=0.1)
linewidth = 3.0
fontsize_sec = 14
'''
the prediction is implemented through for loop as it is how the data is obtained in the real AFP process
input data is received in chuncks as tape moves
'''
frames_to_pred_total = number_of_frames-frames_to_predict
frames_to_pred_total = 70
for i in range(frames_to_pred_total):
    #reset lstm state to get independent prediction between timesteps (batches)
    model_lstm.reset_states()
    print(f'\n\n\nStep {i+1}')
    #get ground truth window
    image_data_frames_input = image_data_frames_gt[i:window_size+i]
    print(f'Input shape: {image_data_frames_input.shape}')
    
    print('\n##########Encoding started#############')
    image_data_frames_encoded = model_encoder.predict(image_data_frames_input)
    print('##########Encoding finished############')
    print(f'Encoded shape: {image_data_frames_encoded.shape}')
    
    print('\n##########LSTM prediction started#############')
    image_data_frames_lstm = np.copy(image_data_frames_encoded)
    lstm_input = image_data_frames_lstm
    #predict
    print(f'Input shape: {lstm_input.shape}')
    predicted_sequence = model_lstm.predict(lstm_input[np.newaxis])
    print(predicted_sequence.shape)
    image_data_frames_lstm = np.concatenate((image_data_frames_lstm, predicted_sequence[0]), axis=0)

    print(f'Output shape: {image_data_frames_lstm.shape}')
    print('##########LSTM prediction finished#############')

    print('\n##########Decoding started#############')
    image_data_frames_decoded = model_decoder.predict(image_data_frames_lstm)
    print('##########Decoding finished############')
    print(f'Decoded shape: {image_data_frames_decoded.shape}')

    #assemble the whole image from frames
    image_data_predicted = assemble_image(image_data_frames_decoded)
  
    #plot
    inner = gridspec.GridSpecFromSubplotSpec(
        numb_plots_per_frame, 1,
        subplot_spec=outer[i], wspace=0.1, hspace=0.1)

    #plot ground truth
    ax = plt.Subplot(fig, inner[0])
    x_lim = image_data_gt.shape[1]
    #plot only a portion of gt for comparison
    index_current = window_size*stride_step+stride_step*i+frames_to_predict*stride_step
    ax.imshow(image_data_gt[:,:index_current,:])
    ax.set_xlim(0, x_lim)
    ax.set_title(f'Step {i+1}, Ground truth')
    #plot separator input/prediction
    x_separator = window_size*stride_step+stride_step*i
    ax.axvline(x = x_separator, color = 'red')
    fig.add_subplot(ax)
    
    #plot prediction
    ax = plt.Subplot(fig, inner[1])
    #reassemble to include all previous gt
    image_data_predicted = np.concatenate(
        (
            image_data_gt[:,:x_separator,:],
            image_data_predicted[:,-frames_to_predict*stride_step:,:]
        ),
        axis=1
    )
    ax.imshow(image_data_predicted)
    ax.set_title(f'LSTM prediction')
    ax.set_xlim(0, x_lim)
    ax.axvline(x = x_separator, color = 'red')
    fig.add_subplot(ax)
    
    if 1:
        #get errors
        image_gt_1_frames = image_data_frames_gt[i+window_size:i+window_size+frames_to_predict]
        image_pred_2_frames = image_data_frames_decoded[window_size:]

        ssim_frames, mse_gray_frames, mse_rgb_frames = get_errors(
            image_gt_1_frames, 
            image_pred_2_frames
                )   
        #plot errors
        
        
        ax = plt.Subplot(fig, inner[2])
        ax.set_title(f'Errors')   
        ax.plot(mse_gray_frames, label='MSE grayscale')
        ax.plot(mse_rgb_frames, label='MSE RGB')
        color = 'tab:purple'
        ax_add = ax.twinx()
        ax_add.plot(ssim_frames, color=color, label='SSIM')
        ax_add.set_ylabel('SSIM', color=color)
        ax_add.tick_params(axis='y', labelcolor=color)
        lines, labels = ax.get_legend_handles_labels()
        lines2, labels2 = ax_add.get_legend_handles_labels()
        ax.legend(lines + lines2, labels + labels2)
        x_lim = number_of_frames
        ax.set_xlim(0, x_lim)
        fig.add_subplot(ax)
        fig.add_subplot(ax_add)


In [ ]:
#unit 5
numb_cols = 3
numb_plots_per_frame = 3

number_of_frames = image_data_frames_gt.shape[0]
numb_rows = int(np.ceil(number_of_frames/numb_cols))

model_encoder = model_ed_list[0]
model_decoder = model_ed_list[1]


fig = plt.figure(figsize=(20, 15*numb_rows))
outer = gridspec.GridSpec(numb_rows, numb_cols, wspace=0.2, hspace=0.1)
linewidth = 3.0
fontsize_sec = 14
'''
the prediction is implemented through for loop as it is how the data is obtained in the real AFP process
input data is received in chuncks as tape moves
'''
frames_to_pred_total = number_of_frames-frames_to_predict
frames_to_pred_total = 70
for i in range(frames_to_pred_total):
    #reset lstm state to get independent prediction between timesteps (batches)
    model_lstm.reset_states()
    print(f'\n\n\nStep {i+1}')
    #get ground truth window
    image_data_frames_input = image_data_frames_gt[i:window_size+i]
    print(f'Input shape: {image_data_frames_input.shape}')
    
    print('\n##########Encoding started#############')
    image_data_frames_encoded = model_encoder.predict(image_data_frames_input)
    print('##########Encoding finished############')
    print(f'Encoded shape: {image_data_frames_encoded.shape}')
    
    print('\n##########LSTM prediction started#############')
    image_data_frames_lstm = np.copy(image_data_frames_encoded)
    lstm_input = image_data_frames_lstm
    #predict
    print(f'Input shape: {lstm_input.shape}')
    predicted_sequence = model_lstm.predict(lstm_input[np.newaxis])
    print(predicted_sequence.shape)
    image_data_frames_lstm = np.concatenate((image_data_frames_lstm, predicted_sequence[0]), axis=0)

    print(f'Output shape: {image_data_frames_lstm.shape}')
    print('##########LSTM prediction finished#############')

    print('\n##########Decoding started#############')
    image_data_frames_decoded = model_decoder.predict(image_data_frames_lstm)
    print('##########Decoding finished############')
    print(f'Decoded shape: {image_data_frames_decoded.shape}')

    #assemble the whole image from frames
    image_data_predicted = assemble_image(image_data_frames_decoded)
  
    #plot
    inner = gridspec.GridSpecFromSubplotSpec(
        numb_plots_per_frame, 1,
        subplot_spec=outer[i], wspace=0.1, hspace=0.1)

    #plot ground truth
    ax = plt.Subplot(fig, inner[0])
    x_lim = image_data_gt.shape[1]
    #plot only a portion of gt for comparison
    index_current = window_size*stride_step+stride_step*i+frames_to_predict*stride_step
    ax.imshow(image_data_gt[:,:index_current,:])
    ax.set_xlim(0, x_lim)
    ax.set_title(f'Step {i+1}, Ground truth')
    #plot separator input/prediction
    x_separator = window_size*stride_step+stride_step*i
    ax.axvline(x = x_separator, color = 'red')
    fig.add_subplot(ax)
    
    #plot prediction
    ax = plt.Subplot(fig, inner[1])
    #reassemble to include all previous gt
    image_data_predicted = np.concatenate(
        (
            image_data_gt[:,:x_separator,:],
            image_data_predicted[:,-frames_to_predict*stride_step:,:]
        ),
        axis=1
    )
    ax.imshow(image_data_predicted)
    ax.set_title(f'LSTM prediction')
    ax.set_xlim(0, x_lim)
    ax.axvline(x = x_separator, color = 'red')
    fig.add_subplot(ax)
    
    if 1:
        #get errors
        image_gt_1_frames = image_data_frames_gt[i+window_size:i+window_size+frames_to_predict]
        image_pred_2_frames = image_data_frames_decoded[window_size:]

        ssim_frames, mse_gray_frames, mse_rgb_frames = get_errors(
            image_gt_1_frames, 
            image_pred_2_frames
                )   
        #plot errors
        
        
        ax = plt.Subplot(fig, inner[2])
        ax.set_title(f'Errors')   
        ax.plot(mse_gray_frames, label='MSE grayscale')
        ax.plot(mse_rgb_frames, label='MSE RGB')
        color = 'tab:purple'
        ax_add = ax.twinx()
        ax_add.plot(ssim_frames, color=color, label='SSIM')
        ax_add.set_ylabel('SSIM', color=color)
        ax_add.tick_params(axis='y', labelcolor=color)
        lines, labels = ax.get_legend_handles_labels()
        lines2, labels2 = ax_add.get_legend_handles_labels()
        ax.legend(lines + lines2, labels + labels2)
        x_lim = number_of_frames
        ax.set_xlim(0, x_lim)
        fig.add_subplot(ax)
        fig.add_subplot(ax_add)


In [ ]:
#unit 5
numb_cols = 3
numb_plots_per_frame = 3

number_of_frames = image_data_frames_gt.shape[0]
numb_rows = int(np.ceil(number_of_frames/numb_cols))

model_encoder = model_ed_list[0]
model_decoder = model_ed_list[1]


fig = plt.figure(figsize=(20, 15*numb_rows))
outer = gridspec.GridSpec(numb_rows, numb_cols, wspace=0.2, hspace=0.1)
linewidth = 3.0
fontsize_sec = 14
'''
the prediction is implemented through for loop as it is how the data is obtained in the real AFP process
input data is received in chuncks as tape moves
'''
frames_to_pred_total = number_of_frames-frames_to_predict
frames_to_pred_total = 70
for i in range(frames_to_pred_total):
    #reset lstm state to get independent prediction between timesteps (batches)
    model_lstm.reset_states()
    print(f'\n\n\nStep {i+1}')
    #get ground truth window
    image_data_frames_input = image_data_frames_gt[i:window_size+i]
    print(f'Input shape: {image_data_frames_input.shape}')
    
    print('\n##########Encoding started#############')
    image_data_frames_encoded = model_encoder.predict(image_data_frames_input)
    print('##########Encoding finished############')
    print(f'Encoded shape: {image_data_frames_encoded.shape}')
    
    print('\n##########LSTM prediction started#############')
    image_data_frames_lstm = np.copy(image_data_frames_encoded)
    lstm_input = image_data_frames_lstm
    #predict
    print(f'Input shape: {lstm_input.shape}')
    predicted_sequence = model_lstm.predict(lstm_input[np.newaxis])
    print(predicted_sequence.shape)
    image_data_frames_lstm = np.concatenate((image_data_frames_lstm, predicted_sequence[0]), axis=0)

    print(f'Output shape: {image_data_frames_lstm.shape}')
    print('##########LSTM prediction finished#############')

    print('\n##########Decoding started#############')
    image_data_frames_decoded = model_decoder.predict(image_data_frames_lstm)
    print('##########Decoding finished############')
    print(f'Decoded shape: {image_data_frames_decoded.shape}')

    #assemble the whole image from frames
    image_data_predicted = assemble_image(image_data_frames_decoded)
  
    #plot
    inner = gridspec.GridSpecFromSubplotSpec(
        numb_plots_per_frame, 1,
        subplot_spec=outer[i], wspace=0.1, hspace=0.1)

    #plot ground truth
    ax = plt.Subplot(fig, inner[0])
    x_lim = image_data_gt.shape[1]
    #plot only a portion of gt for comparison
    index_current = window_size*stride_step+stride_step*i+frames_to_predict*stride_step
    ax.imshow(image_data_gt[:,:index_current,:])
    ax.set_xlim(0, x_lim)
    ax.set_title(f'Step {i+1}, Ground truth')
    #plot separator input/prediction
    x_separator = window_size*stride_step+stride_step*i
    ax.axvline(x = x_separator, color = 'red')
    fig.add_subplot(ax)
    
    #plot prediction
    ax = plt.Subplot(fig, inner[1])
    #reassemble to include all previous gt
    image_data_predicted = np.concatenate(
        (
            image_data_gt[:,:x_separator,:],
            image_data_predicted[:,-frames_to_predict*stride_step:,:]
        ),
        axis=1
    )
    ax.imshow(image_data_predicted)
    ax.set_title(f'LSTM prediction')
    ax.set_xlim(0, x_lim)
    ax.axvline(x = x_separator, color = 'red')
    fig.add_subplot(ax)
    
    if 1:
        #get errors
        image_gt_1_frames = image_data_frames_gt[i+window_size:i+window_size+frames_to_predict]
        image_pred_2_frames = image_data_frames_decoded[window_size:]

        ssim_frames, mse_gray_frames, mse_rgb_frames = get_errors(
            image_gt_1_frames, 
            image_pred_2_frames
                )   
        #plot errors
        
        
        ax = plt.Subplot(fig, inner[2])
        ax.set_title(f'Errors')   
        ax.plot(mse_gray_frames, label='MSE grayscale')
        ax.plot(mse_rgb_frames, label='MSE RGB')
        color = 'tab:purple'
        ax_add = ax.twinx()
        ax_add.plot(ssim_frames, color=color, label='SSIM')
        ax_add.set_ylabel('SSIM', color=color)
        ax_add.tick_params(axis='y', labelcolor=color)
        lines, labels = ax.get_legend_handles_labels()
        lines2, labels2 = ax_add.get_legend_handles_labels()
        ax.legend(lines + lines2, labels + labels2)
        x_lim = number_of_frames
        ax.set_xlim(0, x_lim)
        fig.add_subplot(ax)
        fig.add_subplot(ax_add)
